# Neural Networks for Data Science Applications
## End-of-term homework: Advanced transfer learning

**Name**: Federico Argilli

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# To ensure reproducible results (as much as possible)
tf.keras.utils.set_random_seed(1234)

## Data loading

I am a passionate supporter of online gaming, and one relevant issue within the online gaming community is the toxicity exhibited by some players. Addressing this problem is indeed a challenging task.

As I will show in the model section, identifying toxic behavior is difficult with standard sentiment analysis. This is because certain sentences flagged as negative may actually make complete sense within the context of the game.

The dataset selected for this project consists of an extensive collection of chat logs from one of the most famous multiplayer online games: League of Legends. It is available on [Kaggle](https://www.kaggle.com/datasets/simshengxue/league-of-legends-tribunal-chatlogs) and originates from a platform provided by the game's company, which made these logs accessible.

In [ ]:
from google.colab import files
import pandas as pd
import json

#### Uncomment to setup kaggle key
files.upload()  # upload kaggle.json
! mkdir -p ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# download and extract data
!kaggle datasets download -d simshengxue/league-of-legends-tribunal-chatlogs
!unzip -q league-of-legends-tribunal-chatlogs.zip

 97% 81.0M/83.8M [00:01<00:00, 91.9MB/s]
100% 83.8M/83.8M [00:01<00:00, 80.1MB/s]


In [ ]:
# loading the data as a pandas DataFrame
with open("input-data.json","r") as f:
  raw_data = json.load(f)

In [ ]:
# dropping non-relevant columns
df = pd.DataFrame(raw_data["Messages"])
df.drop(["IsAllChat","IsAlly","ReferredChampions"],axis=1, inplace=True)

The relevant columns for this project are:
- `IsToxic`: wheter the text is toxic or not, flagged by human control
- `Content`: the actual text message




In [ ]:
df.head(10)

,IsToxic,Content
0,False,gold 2 zed
1,False,IIII
2,False,nice premade lie :o
3,False,ISI
4,False,smiteless pls
5,False,smiteless pls
6,False,riven?
7,False,report top no help jnh
8,False,warded there
9,False,K


#### Preprocess, test and train
The model (T5) was trained on the SST2 dataset, thus for sentiment classification we can use the prefix `"sst2 sentence:"`.\
In order to create the training and test datasets we have to concatenate the prefix to all sentences and transform the `IsToxic` column with the actual labels to predict: positive or negative.


In [ ]:
task_prefix = "sst2 sentence: "  # equivalent to say "sentiment analysis sentence:"
df["Sentence"] = task_prefix + df["Content"].map(str)   # insert the prefix

# create a column with the actual predicted label
df["label"] = df["IsToxic"].apply(lambda x: "negative" if x else "positive")  # if toxis --> "negative", if not toxis --> positive

We can see below the final version of the dataset that will be later tokenized in order to train the model.

In [ ]:
df[["Sentence","label"]].head()

,Sentence,label
0,sst2 sentence: gold 2 zed,positive
1,sst2 sentence: IIII,positive
2,sst2 sentence: nice premade lie :o,positive
3,sst2 sentence: ISI,positive
4,sst2 sentence: smiteless pls,positive


In [ ]:
# split into train and test
train_df = df.sample(n=15000, random_state=123)
test_df = df.drop(train_df.index).sample(n=2000, random_state=123)

### Advanced transfer learning

**DESCRIPTION OF THE CODE**

The model I chose is Google's [T5](https://arxiv.org/abs/1910.10683). It adheres to the standard encoder-decoder architecture and operates as a multi-task model in the text-to-text format.\
I opted for the "small" version (60M parameters) as it seemed like the most practical choice for efficient training and execution on limited resources. Despite its smaller size, it still managed to yield good results after the fine-tuning process.

I aimed to fine-tune the model using LoRA's approach. Following the LoRA [paper's](https://arxiv.org/abs/2106.09685) recommendations, I applied LoRA to the $q$ and $v$ matrices in the attention layers, both within the encoder and decoder of the model.

The `transformer` module is used solely for downloading the model from [Hugging Face](https://huggingface.co/google-t5/t5-small). Everything else, from generation to defining LoRA components and training, is accomplished in TensorFlow.

In [ ]:
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration
import transformers

tokenizer = T5TokenizerFast.from_pretrained("t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")  # load the model with tensorflow

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


An example of what I anticipated above.\
Here there are some example sentences that are not toxic in the context of the game but are miss-classfied by the model:

In [ ]:
task_prefix = "sst2 sentence: "

sentences = ["We need more need kills", "ROFL", "hard focus the enemy"]

# append the prefix to each sentence and return the tokenized ids
inputs = tokenizer([task_prefix + sentence for sentence in sentences], return_tensors="tf", padding=True)

# generate the response from the model
output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
)

labels = tokenizer.batch_decode(output_sequences, skip_special_tokens=True) # decode the response

print(*zip(sentences,labels))

('We need more need kills', 'negative') ('ROFL', 'positive') ('hard focus the enemy', 'negative')


#### LoRA model
I had to define a new type of layer (`LoraDense`) in order to inject low rank matrices in the model.
The `dense` attribute of this new model represent the orginal dense layer of $q$ or $v$ in the attention layers.

I also had to create a model-trainer subclass to customize the fit method.

In [ ]:
# tensorflow implementation of the pytorch code from the orginal paper
# https://github.com/microsoft/LoRA/blob/main/loralib/layers.py
class LoraDense(tf.keras.layers.Layer):

  def __init__(self, config, rank=8, **kwargs):
    super().__init__(**kwargs)
    self.dense_config = config      # initialize and copy the original dense
    self.dense = tf.keras.layers.Dense.from_config(self.dense_config)
    self.dense.trainable = False   # set the original dense layer as non trainable

    self.r = rank

  def build(self, dense_build_config, weights):
    in_dim = weights[0].shape[0]
    out_dim = weights[0].shape[1]

    self.dense.build_from_config(dense_build_config)
    self.dense.set_weights(weights)          # copy the original weight of the model

    # Add low rank matrices
    self.B = self.add_weight(shape=(in_dim,self.r),
                              initializer='zeros',    # B initialized with all zeros
                              trainable=True,
                              name="Lora_B")
    self.A = self.add_weight(shape=(self.r,out_dim),
                              initializer='random_normal',   # A initialized at random
                              trainable=True,
                              name="Lora_A")
    self.built = True

  def call(self,x):
    W = self.dense(x)  # orginal output
    BA = self.B @ self.A

    return W + x@BA  # modified output

In [ ]:
# https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
class LoraCustomTrainer(TFT5ForConditionalGeneration):
  def __init__(self, *args, **kwargs):
    super().__init__(*args,**kwargs)

  @tf.function
  def train_step(self,data):
    x,y = data

    with tf.GradientTape() as tape:
      loss = self(input_ids=x, labels = y).loss # compue the loss

    trainable_variables = []

    # we need to iterate over all the layers because model.trainable_weights does not return weights
    for l in self.encoder.submodules:
      if isinstance(l,transformers.models.t5.modeling_tf_t5.TFT5Attention):
        trainable_variables += l.q.trainable_weights  # add lora parameters from q
        trainable_variables += l.v.trainable_weights  # add lora parameters from v

    for l in self.decoder.submodules:
      if isinstance(l,transformers.models.t5.modeling_tf_t5.TFT5Attention): # same cycle on the decoder
        trainable_variables += l.q.trainable_weights
        trainable_variables += l.v.trainable_weights

    grads = tape.gradient(loss, trainable_variables)

    self.optimizer.apply_gradients(zip(grads, trainable_variables))  # apply gradients

    return {"loss":loss}


In [ ]:
trainer = LoraCustomTrainer.from_pretrained("t5-small")

All PyTorch model weights were used when initializing LoraCustomTrainer.

All the weights of LoraCustomTrainer were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LoraCustomTrainer for predictions without further training.


Below I replace the model's layers with the new LoRA layers and initialize them with the original configuration and weigths.

In [ ]:
# injection of LoraDense layers
for l in trainer.encoder.submodules + trainer.decoder.submodules:
  l.trainable = False  # mark all layers as not trainable

  if isinstance(l,transformers.models.t5.modeling_tf_t5.TFT5Attention):  # select only attention layers

    # get original dense layer configuration and weights
    build_config_q, build_config_v = l.q.get_build_config(), l.v.get_build_config()
    config_q, config_v = l.q.get_config(), l.v.get_config()
    weights_q, weights_v = l.q.get_weights(), l.v.get_weights()

    l.q = LoraDense(config_q)  # replace the dense layer for q in the original attention layer
    l.q.build(build_config_q, weights_q)

    l.v = LoraDense(config_v)  # replace the dense layer for v in the original attention layer
    l.v.build(build_config_v, weights_v)

#### Training and results

In [ ]:
# mark the embeddings as not trainable
trainer.shared.trainable=False

# check that only the LoRA matrices are trainable weights
for l in trainer.encoder.submodules:
  if isinstance(l,transformers.models.t5.modeling_tf_t5.TFT5Attention):
    print([weight.name for weight in l.q.trainable_weights])
    print([weight.name for weight in l.v.trainable_weights])

['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']
['Lora_B:0', 'Lora_A:0']


In [ ]:
# tokenize the training dataset
X_train = tokenizer(train_df["Sentence"].to_list(), return_tensors="tf", padding=True).input_ids
y_train = tokenizer(train_df["label"].to_list(), return_tensors="tf",padding=True).input_ids

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.1)
trainer.compile(optimizer=opt)

In [ ]:
trainer.fit(X_train,y_train, epochs=3)

Epoch 1/3
469/469 [==============================] - 89s 138ms/step - loss: 0.2609
Epoch 2/3
469/469 [==============================] - 63s 135ms/step - loss: 0.2338
Epoch 3/3
469/469 [==============================] - 62s 133ms/step - loss: 0.2334


Below we can check the results on the test set.

In [ ]:
# tokenize the test dataset
inputs = tokenizer(test_df["Sentence"].to_list(), return_tensors="tf", padding=True)
label = tokenizer(test_df["label"].to_list(), return_tensors="tf",padding=True)

# generate with the fine-tuned model
lora_output_sequences = trainer.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

# generate with the original model
base_output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

In [ ]:
new_acc = sum(test_df["label"] == tokenizer.batch_decode(lora_output_sequences, skip_special_tokens=True))/len(test_df)
old_acc = sum(test_df["label"] == tokenizer.batch_decode(base_output_sequences, skip_special_tokens=True))/len(test_df)

print(f"Accuracy of the finetuned model: {new_acc}")
print(f"Accuracy of the base model: {old_acc}")

Accuracy of the finetuned model: 0.8055
Accuracy of the base model: 0.5145


The fine-tuned model demonstrated a substantial improvement over the base model's 50% accuracy, with the training of only a small fraction of parameters using LoRA.\
While there is room for enhancements, this result remains remarkable, particularly considering the modest size of the model.